<a href="https://colab.research.google.com/github/luiseufrasio/safe-route-ai/blob/main/safe_route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Instalando o SDK do Google

In [1]:
!pip install -q -U google-generativeai

###Configurando a API KEY do Google Gemini

In [2]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get("SECRET_KEY"))

###Criando a lista de mapas e o método para upload a partir do caminho no Colab

In [3]:
from pathlib import Path
import hashlib

uploaded_maps = []
def upload_if_needed(pathname: str) -> list[str]:
  path = Path(pathname)
  hash_id = hashlib.sha256(path.read_bytes()).hexdigest()
  try:
    existing_file = genai.get_file(name=hash_id)
    return [existing_file]
  except:
    pass
  uploaded_maps.append(genai.upload_file(path=path, display_name=hash_id))
  return [uploaded_maps[-1]]

#Carregando o mapa topográfico do Estado do RS
##### Fonte: https://pt-br.topographic-map.com/map-pb651/Rio-Grande-do-Sul/

In [4]:
rs_input_parts = ["input: ", *upload_if_needed("/content/RS-Metropolitana.png"), "Analise esse mapa TOPOGRÁFICO de cores do Rio Grande de Sul, tente identificar as cidades que estão nele e me dê sua altitude aproximada com base nas cores. Mostre o resultado ordenado pela altitude encontrada."]

#Setup do Modelo

In [5]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

#Executando a pesquisa e mostrando a resposta do modelo

In [6]:
response1 = model.generate_content(rs_input_parts)

In [7]:
from IPython.display import Markdown

display(Markdown(response1.text))

## Análise do Mapa Topográfico do Rio Grande do Sul:

Com base na análise do mapa topográfico fornecido, identifiquei as seguintes cidades e suas altitudes aproximadas, ordenadas da maior para a menor altitude:

**Cidade** | **Altitude Aproximada (m)** | **Cor no Mapa**
------- | -------- | --------
Torres | 1049 | Vermelho escuro 
São Francisco de Paula | 967 | Vermelho
Canela | ~967 | Vermelho
Três Coroas | ~887 | Vermelho claro
Três Cachoeiras | ~809 | Laranja escuro
Arroio do Sal | ~734 | Laranja
Nova Petrópolis | ~662 | Amarelo escuro
Gravataí | ~592 | Amarelo esverdeado
São Leopoldo | ~524 | Verde claro
Novo Hamburgo | ~524 | Verde claro
Cachoeirinha | ~459 | Verde azulado
Canoas | ~459 | Verde azulado
Viamão | ~397 | Azul claro
Porto Alegre | ~339 | Azul
Osório | ~283 | Azul escuro
Tramandaí | ~231 | Azul escuro
Cidreira | ~182 | Azul muito escuro
Balneário Pinhal | ~137 | Azul muito escuro
Palmares do Sul | ~97 | Azul muito escuro próximo ao ciano
Barra do Ribeiro | ~61 | Ciano
Guaíba | ~31 | Ciano claro
Lagoa do Casamento | ~8 | Azul muito claro

**Observações:**

* As altitudes são aproximadas, baseadas na escala de cores do mapa. A precisão é limitada pela resolução do mapa e pela variação de altitude dentro de cada cidade. 
* Cidades menores e com menor variação de altitude podem ser difíceis de analisar com precisão.
* A lista inclui apenas algumas das cidades presentes no mapa, focando nas mais visíveis e com altitudes mais distintas.

Espero que esta análise seja útil! 😊 


#Perguntando a Pessoa qual cidade ela está

In [8]:
lista_cidades = ['Canoas', 'Lajeado', 'Pelotas', 'Porto Alegre', 'Rio Grande', 'São Leopoldo']
cidade = input("Em qual dessas cidades você está: [" + ", ".join(lista_cidades) + "] ?\t")

while cidade not in lista_cidades:
  print("A cidade escolhida não está na lista. Escolha novamente...")
  cidade = input("[" + ", ".join(lista_cidades) + "] \t")

Em qual dessas cidades você está: [Canoas, Lajeado, Pelotas, Porto Alegre, Rio Grande, São Leopoldo] ?	Porto Alegre


#Saindo da cidade escolhida para cidades com altitude acima dos 100m

In [9]:
destinos = ['Torres', 'São Francisco de Paula', 'Canela', 'Três Coroas', 'Igrejinha', 'Taquara', 'Rolante', 'Osório', 'Santo Antônio da Patrulha', 'Tramandaí']
cidade_input = "Trace rotas partindo da cidade " + cidade + " no estado do Rio Grande do Sul para 3 cidades nessa lista: [" + ", ".join(destinos) + "]. Evite caminhos que possam passar por pontes."

response2 = model.generate_content(cidade_input)

display(Markdown(response2.text))

## Rotas a partir de Porto Alegre sem pontes:

**Observação:** Evitar completamente pontes em algumas rotas é desafiador devido à geografia da região. As rotas abaixo minimizam o uso de pontes, mas pode ser impossível evitá-las completamente em certos trechos. 

**1. Porto Alegre - Torres:**

* **Sem passar por pontes (impossível):** A rota mais direta para Torres a partir de Porto Alegre cruza a Ponte do Guaíba. É impossível chegar a Torres sem cruzar pelo menos uma ponte.
* **Minimizando pontes:**
    * Seguir pela BR-116 até a RS-040 em Viamão.
    * Continuar pela RS-040 até Santo Antônio da Patrulha e seguir pela RS-486 (Rota do Sol) até Torres.
    * **Observação:** Haverá pontes menores ao longo da RS-040 e RS-486.

**2. Porto Alegre - São Francisco de Paula:**

* **Sem passar por pontes (impossível):** As principais rotas para São Francisco de Paula cruzam a Ponte do Guaíba.
* **Minimizando pontes:**
    * Seguir pela BR-116 até a RS-020 em São Leopoldo.
    * Seguir pela RS-020, passando por Taquara, até chegar a São Francisco de Paula. 
    * **Observação:** Haverá pontes menores ao longo da RS-020.

**3. Porto Alegre - Tramandaí:**

* **Sem passar por pontes (impossível):**  A rota mais direta para Tramandaí cruza a Ponte do Guaíba.
* **Minimizando pontes:**
    * Seguir pela BR-116 até a RS-040 em Viamão.
    * Continuar pela RS-040 até Santo Antônio da Patrulha e seguir pela RS-786 até Tramandaí.
    * **Observação:** Haverá pontes menores ao longo da RS-040 e RS-786.

**Recomendações:**

* Utilize um aplicativo de GPS como o Google Maps ou Waze para visualizar as rotas e possíveis pontes ao longo do caminho.
* Ajuste as rotas conforme necessário, buscando alternativas que minimizem o cruzamento de pontes.
* Esteja ciente de que, em alguns casos, evitar pontes completamente pode aumentar significativamente a distância e o tempo de viagem. 

**Lembre-se:**  As condições das estradas e as rotas disponíveis podem mudar. Consulte informações atualizadas antes de iniciar sua viagem.

#Chat Final para mais dúvidas

In [17]:
chat = model.start_chat(history=[
    {"role": "user", "parts": [rs_input_parts[2]]},
    {"role": "model", "parts": [response1.text]},
    {"role": "user", "parts": [cidade_input]},
    {"role": "model", "parts": [response2.text]}])
sair = "[digite 'sair' para finalizar a conversa] "
prompt = input("Ajudo em algo mais? " + sair)

while prompt != 'sair':
  response3 = chat.send_message(prompt)
  print("Modelo: ")
  display(Markdown(response3.text))

  chat.history.append({"role": "user", "parts": [prompt]})
  chat.history.append({"role": "model", "parts": [response3.text]})

  print("[digite 'sair' para finalizar a conversa]\n")
  prompt = input("Você: ")

Ajudo em algo mais? [digite 'sair' para finalizar a conversa] Consegue mostrar no google maps a rota de Porto Alegre para Torres?
Modelo: 


Infelizmente, eu não consigo interagir diretamente com o Google Maps. Sou um modelo de linguagem e não tenho a capacidade de controlar aplicativos externos. 

Para visualizar a rota de Porto Alegre para Torres no Google Maps, siga estes passos:

1. **Abra o Google Maps:** Acesse o site [https://www.google.com.br/maps](https://www.google.com.br/maps) ou abra o aplicativo em seu celular.
2. **Digite os locais:** No campo de busca, digite "Porto Alegre" e, em seguida, "Torres, RS".
3. **Selecione a rota:** O Google Maps mostrará a rota mais rápida entre as duas cidades. Você pode clicar em "Opções de rota" para personalizar o trajeto, como evitar pedágios ou balsas.

Espero que isso ajude! 😊 


[digite 'sair' para finalizar a conversa]

Você: sair
